# Imports

In [1]:
import pandas as pd
import numpy as np

# Loading data and Creating Windows

In [2]:
# Load the provided CSV file
data = pd.read_csv('../Dataset/Physiology.csv')
data['date'] = pd.to_datetime(data['date'])

data

,patient_id,date,device_type,value,unit
0,0697d,2019-06-28 13:42:09,Body Temperature,36.072,Cel
1,0697d,2019-06-28 13:42:44,Body Temperature,35.831,Cel
2,0697d,2019-06-28 13:43:40,Body Temperature,35.831,Cel
3,0697d,2019-06-28 13:45:15,Systolic blood pressure,165.000,mm[Hg]
4,0697d,2019-06-28 13:45:15,Diastolic blood pressure,82.000,mm[Hg]
...,...,...,...,...,...
17674,fd100,2019-06-28 21:09:27,Heart rate,61.000,beats/min
17675,fd100,2019-06-28 21:32:19,Body Temperature,36.879,Cel
17676,fd100,2019-06-28 21:32:46,Body Temperature,36.879,Cel
17677,fd100,2019-06-29 21:05:11,Body Temperature,36.240,Cel


In [3]:
data.dtypes

patient_id             object
date           datetime64[ns]
device_type            object
value                 float64
unit                   object
dtype: object

In [4]:
# Exclude rows where device_type is 'O/E - muscle mass', 'Skin Temperature', or 'Total Body Water'
data = data[~data['device_type'].isin(['O/E - muscle mass', 'Skin Temperature', 'Total body water'])]

In [5]:
def assign_time_window(row):
    hour = row.hour
    minute = row.minute

    # The first window is based on the hour itself
    start_hour_1 = hour
    end_hour_1 = (start_hour_1 + 1) % 24  # Wrapping around midnight

    # The second window depends on whether the minute is >= 30
    if minute >= 30:
        start_hour_2 = hour
    else:
        start_hour_2 = hour - 1 if hour > 0 else 23  # Wrapping around midnight

    end_hour_2 = (start_hour_2 + 1) % 24

    # Formatting the time windows
    window1 = f"{start_hour_1:02d}:00-{end_hour_1:02d}:00"
    window2 = f"{start_hour_2:02d}:30-{end_hour_2:02d}:30"

    return [window1, window2]

# Applying the function to each row and expanding the results into two columns
data[['window_1', 'window_2']] = pd.DataFrame(data['date'].apply(assign_time_window).tolist(), index=data.index)

data.head()

C:\Users\ACER\AppData\Local\Temp\ipykernel_7836\2230207906.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['window_1', 'window_2']] = pd.DataFrame(data['date'].apply(assign_time_window).tolist(), index=data.index)
C:\Users\ACER\AppData\Local\Temp\ipykernel_7836\2230207906.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['window_1', 'window_2']] = pd.DataFrame(data['date'].apply(assign_time_window).tolist(), index=data.index)


,patient_id,date,device_type,value,unit,window_1,window_2
0,0697d,2019-06-28 13:42:09,Body Temperature,36.072,Cel,13:00-14:00,13:30-14:30
1,0697d,2019-06-28 13:42:44,Body Temperature,35.831,Cel,13:00-14:00,13:30-14:30
2,0697d,2019-06-28 13:43:40,Body Temperature,35.831,Cel,13:00-14:00,13:30-14:30
3,0697d,2019-06-28 13:45:15,Systolic blood pressure,165.000,mm[Hg],13:00-14:00,13:30-14:30
4,0697d,2019-06-28 13:45:15,Diastolic blood pressure,82.000,mm[Hg],13:00-14:00,13:30-14:30


In [6]:
# Function to convert window and date into a datetime object with the start time of the window
def window_to_datetime(row, window_col):
    # Extracting date and start time from the row
    date = row['date'].date()
    start_time = row[window_col].split('-')[0]

    # Combining date and start time into a datetime object
    return pd.to_datetime(f"{date} {start_time}")

# Applying the function to convert window_1 and window_2
data['window_1'] = data.apply(window_to_datetime, window_col='window_1', axis=1)
data['window_2'] = data.apply(window_to_datetime, window_col='window_2', axis=1)

# Display the first 20 rows with the updated columns
data.head()

C:\Users\ACER\AppData\Local\Temp\ipykernel_7836\1774452290.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['window_1'] = data.apply(window_to_datetime, window_col='window_1', axis=1)
C:\Users\ACER\AppData\Local\Temp\ipykernel_7836\1774452290.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['window_2'] = data.apply(window_to_datetime, window_col='window_2', axis=1)


,patient_id,date,device_type,value,unit,window_1,window_2
0,0697d,2019-06-28 13:42:09,Body Temperature,36.072,Cel,2019-06-28 13:00:00,2019-06-28 13:30:00
1,0697d,2019-06-28 13:42:44,Body Temperature,35.831,Cel,2019-06-28 13:00:00,2019-06-28 13:30:00
2,0697d,2019-06-28 13:43:40,Body Temperature,35.831,Cel,2019-06-28 13:00:00,2019-06-28 13:30:00
3,0697d,2019-06-28 13:45:15,Systolic blood pressure,165.000,mm[Hg],2019-06-28 13:00:00,2019-06-28 13:30:00
4,0697d,2019-06-28 13:45:15,Diastolic blood pressure,82.000,mm[Hg],2019-06-28 13:00:00,2019-06-28 13:30:00


In [21]:
# Re-attempting to extract time and find min/max
data['time'] = pd.to_datetime(data['date']).dt.time
data['time'].sort_values().max()
# # Finding the minimum and maximum time
# min_time = data['time_from_window_1'].min()
# max_time = data['time_from_window_1'].max()
# data.loc[12527]
# min_time, max_time

C:\Users\ACER\AppData\Local\Temp\ipykernel_7836\1134222949.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['time'] = pd.to_datetime(data['date']).dt.time


datetime.time(23, 54, 12)

In [42]:
data.to_csv('../Dataset/Physiology_Cleaned.csv', index=False)

# Creating Aggregated Physiology Dataset

In [53]:
data = pd.read_csv('../Dataset/Physiology_Cleaned.csv')
data['date'] = pd.to_datetime(data['date'])

data_2 = data.copy()

In [54]:
# Adjust the mapping function again to account for the correct labeling of "O/E - muscle mass"
def map_device_type_to_column(device_type):
    mapping = {
        'Body Temperature': 'Body Temperature',
        'Systolic blood pressure': 'Systolic Blood Pressure',
        'Diastolic blood pressure': 'Diastolic Blood Pressure',
        'Heart rate': 'Heart Rate',
        'Body weight': 'Body weight'
    }
    return mapping.get(device_type, None)

# Apply the mapping to the dataset
data_2['Final Measurement'] = data_2['device_type'].apply(map_device_type_to_column)

averages_window_1 = data_2.groupby(['patient_id', 'Final Measurement', 'window_1'])['value'].mean().reset_index()
averages_window_2 = data_2.groupby(['patient_id', 'Final Measurement', 'window_2'])['value'].mean().reset_index()

averages_window_1.rename(columns={'window_1': 'Window', 'value': 'Average'}, inplace=True)
averages_window_2.rename(columns={'window_2': 'Window', 'value': 'Average'}, inplace=True)

# Combine the two sets of averages
averages_combined = pd.concat([averages_window_1, averages_window_2])

# Pivot the table to have one row per patient_id and window, and columns for each measurement
final_data = averages_combined.pivot_table(index=['patient_id', 'Window'], columns='Final Measurement', values='Average', aggfunc='first').reset_index()

# Ensure all expected columns are present
expected_columns = ['patient_id', 'Window', 'Body Temperature', 'Diastolic Blood Pressure', 'Heart Rate', 'Body weight', 'Systolic Blood Pressure']
final_data = final_data.reindex(columns=expected_columns)

In [55]:
final_data.head()

Final Measurement,patient_id,Window,Body Temperature,Diastolic Blood Pressure,Heart Rate,Body weight,Systolic Blood Pressure
0,0697d,2019-06-28 13:00:00,35.911333,82.0,42.0,NaN,165.0
1,0697d,2019-06-28 13:30:00,36.186400,82.0,42.0,NaN,165.0
2,0697d,2019-06-28 14:00:00,36.599000,NaN,NaN,86.3,NaN
3,0697d,2019-06-28 14:30:00,NaN,NaN,NaN,86.3,NaN
4,0697d,2019-06-29 16:30:00,37.257000,NaN,NaN,NaN,NaN


In [56]:
final_data.shape

(7631, 7)

In [60]:
# Adding the Agitation Labels to aggregated dataset

labels_data = pd.read_csv('..\Dataset\Labels.csv')
labels_data['date'] = pd.to_datetime(labels_data['date'])
agitation_labels = labels_data[labels_data['type'] == 'Agitation']

# Define a function to apply the described logic for setting the "Agitation" column based on the rules provided
def apply_agitation_flag(row, agitation_labels):
    patient_agitations = agitation_labels[agitation_labels['patient_id'] == row['patient_id']]
    
    # Check for any agitation events that affect the current row's window
    for _, agitation in patient_agitations.iterrows():
        window_start = pd.to_datetime(row['Window'])
        
        # Check if agitation is logged at 12pm or 6pm
        if agitation['date'].hour == 18:  # 6pm
            if 16 <= window_start.hour <= 23:
                return 1
        elif agitation['date'].hour == 12:  # 12pm
            if window_start.hour >= 5  or window_start.hour <= 15:
                return 1
                
    # If no conditions met, return 0
    return 0

# Apply the function to the final_corrected_data dataframe to create the "Agitation" column
final_data['Agitation'] = final_data.apply(lambda row: apply_agitation_flag(row, agitation_labels), axis=1)

final_data.head()

Final Measurement,patient_id,Window,Body Temperature,Diastolic Blood Pressure,Heart Rate,Body weight,Systolic Blood Pressure,Agitation
0,0697d,2019-06-28 13:00:00,35.911333,82.0,42.0,NaN,165.0,0
1,0697d,2019-06-28 13:30:00,36.186400,82.0,42.0,NaN,165.0,0
2,0697d,2019-06-28 14:00:00,36.599000,NaN,NaN,86.3,NaN,0
3,0697d,2019-06-28 14:30:00,NaN,NaN,NaN,86.3,NaN,0
4,0697d,2019-06-29 16:30:00,37.257000,NaN,NaN,NaN,NaN,0


In [61]:
final_data[final_data["Agitation"] == 1]

Final Measurement,patient_id,Window,Body Temperature,Diastolic Blood Pressure,Heart Rate,Body weight,Systolic Blood Pressure,Agitation
9,099bc,2019-05-15 11:00:00,NaN,NaN,NaN,53.3,NaN,1
10,099bc,2019-05-15 11:30:00,37.229286,88.0,66.0,53.3,141.0,1
11,099bc,2019-05-15 12:00:00,37.229286,75.5,82.5,NaN,127.0,1
12,099bc,2019-05-15 12:30:00,NaN,63.0,99.0,NaN,113.0,1
13,099bc,2019-05-16 08:30:00,37.045000,78.0,71.0,52.7,152.0,1
...,...,...,...,...,...,...,...,...
7507,eca1f,2019-06-27 09:30:00,36.502000,63.0,89.0,65.3,111.0,1
7508,eca1f,2019-06-28 09:30:00,36.322000,68.0,91.0,65.8,107.0,1
7509,eca1f,2019-06-28 10:00:00,36.322000,68.0,91.0,65.8,107.0,1
7510,eca1f,2019-06-30 14:00:00,36.934000,71.0,87.0,65.7,129.0,1


In [62]:
final_data.to_csv('../Dataset/Physiology_Aggregated.csv', index=False)

# Using LOCF to Fill in NaNs

In [63]:
df = pd.read_csv('../Dataset/Physiology_Aggregated.csv')
df.head()

df['Window'] = pd.to_datetime(df['Window'])
df['day'] = df['Window'].dt.date

In [64]:
df.sort_values(by=['patient_id', 'Window'], inplace=True)

def locf_with_agitation(df):
    # Identify the columns to fill
    columns_to_fill = [col for col in df.columns if col not in ['patient_id', 'Window', 'Agitation','day']]
    
    for col in columns_to_fill:
        # Apply forward fill where 'Agitation' value is the same as in the next row
        df[col] = df.groupby(['patient_id', 'day', 'Agitation'])[col].ffill(limit=2)

    return df

data_filled = df.groupby('patient_id').apply(locf_with_agitation)
data_filled.reset_index(drop=True, inplace=True)
data_filled

,patient_id,Window,Body Temperature,Diastolic Blood Pressure,Heart Rate,Body weight,Systolic Blood Pressure,Agitation,day
0,0697d,2019-06-28 13:00:00,35.911333,82.0,42.0,NaN,165.0,0,2019-06-28
1,0697d,2019-06-28 13:30:00,36.186400,82.0,42.0,NaN,165.0,0,2019-06-28
2,0697d,2019-06-28 14:00:00,36.599000,82.0,42.0,86.3,165.0,0,2019-06-28
3,0697d,2019-06-28 14:30:00,36.599000,82.0,42.0,86.3,165.0,0,2019-06-28
4,0697d,2019-06-29 16:30:00,37.257000,NaN,NaN,NaN,NaN,0,2019-06-29
...,...,...,...,...,...,...,...,...,...
7626,fd100,2019-06-28 20:30:00,36.611500,74.0,61.0,NaN,138.0,0,2019-06-28
7627,fd100,2019-06-28 21:00:00,36.538000,74.0,61.0,NaN,138.0,0,2019-06-28
7628,fd100,2019-06-28 21:30:00,36.879000,74.0,61.0,NaN,138.0,0,2019-06-28
7629,fd100,2019-06-29 20:30:00,36.240000,NaN,NaN,NaN,NaN,0,2019-06-29


In [65]:
data_filled.isna().sum()

patient_id                     0
Window                         0
Body Temperature            1010
Diastolic Blood Pressure    3324
Heart Rate                  3324
Body weight                 3364
Systolic Blood Pressure     3324
Agitation                      0
day                            0
dtype: int64

In [66]:
data_filled = data_filled.dropna()
data_filled[data_filled['Agitation'] == 1]

,patient_id,Window,Body Temperature,Diastolic Blood Pressure,Heart Rate,Body weight,Systolic Blood Pressure,Agitation,day
10,099bc,2019-05-15 11:30:00,37.229286,88.0,66.0,53.3,141.0,1,2019-05-15
11,099bc,2019-05-15 12:00:00,37.229286,75.5,82.5,53.3,127.0,1,2019-05-15
12,099bc,2019-05-15 12:30:00,37.229286,63.0,99.0,53.3,113.0,1,2019-05-15
13,099bc,2019-05-16 08:30:00,37.045000,78.0,71.0,52.7,152.0,1,2019-05-16
14,099bc,2019-05-16 09:00:00,36.794857,78.0,71.0,52.7,152.0,1,2019-05-16
...,...,...,...,...,...,...,...,...,...
7507,eca1f,2019-06-27 09:30:00,36.502000,63.0,89.0,65.3,111.0,1,2019-06-27
7508,eca1f,2019-06-28 09:30:00,36.322000,68.0,91.0,65.8,107.0,1,2019-06-28
7509,eca1f,2019-06-28 10:00:00,36.322000,68.0,91.0,65.8,107.0,1,2019-06-28
7510,eca1f,2019-06-30 14:00:00,36.934000,71.0,87.0,65.7,129.0,1,2019-06-30


In [67]:
data_filled.to_csv('../Dataset/Physiology_Formatted_Dataset_V2.csv', index=False)